# Senate Bill Status

> In the current legislative status field as well as the Legislative history section, there are free text strings that depic actions being taken on these bills, their dates, and the potential actors. However, they are not particularly well-formatted, so we'll need to do some custom string parsing to get this to work.


In [4]:
# | default_exp senate.bill_status
# | export
import re
import datetime

from typing import List, Optional
from nbdev.showdoc import show_doc

from legisph.senate.models import SenateBill, Senator, SenateCommittee

## Parsing Strategy 

Our strategy is to cycle through a predefined list of classes that all have a `parse()` class method which return parsed statuses (if any) and an indicator as to whether to short circuit the function. 

For example, we first define the most common status for "Pending in committee" as follows:

In [5]:
class PendingInCommittee(SenateBill.SenateBillStatus):
    name: str = "Pending in Committee"

    @classmethod
    def parse(cls, h):
        if h.item == "Pending in the Committee":
            return (cls(**h.dict()), False)
        return (None, True)

This is able to parse a status and turn it into the relevant subclass. It returns `False` for the second part of the tuple because if it matches exactly, then there is no further action to take. We also define a convenience function for testing the individual parsing functions:

In [6]:
def test_parsing(statusclass, item):
    parsed = statusclass.parse(
        SenateBill.SenateBillStatus(date=datetime.date.today(), item=item)
    )
    assert parsed[0] is not None
    assert isinstance(parsed[1], bool)
    return parsed

In [7]:
test_parsing(PendingInCommittee, "Pending in the Committee")

(PendingInCommittee(date=datetime.date(2022, 10, 11), item='Pending in the Committee', name='Pending in Committee'),
 False)

This function below then takes a predefined list of classes and cycles through them, chugging out the parsed actions along the way.

In [127]:
# | export
def parse_senate_bill_status(
    status: SenateBill.SenateBillStatus,  # Senate Bill Status to parse into a subclass
    classes: list,  # List of classes through which to cycle
):
    actions = []
    # Split the status into multiple ones if it contains newlines
    statuses = [
        SenateBill.SenateBillStatus(date=status.date, item=s)
        for s in status.item.split(";\n")
    ]
    for s in statuses:
        for c in classes:
            action, cycle = c.parse(s)
            if action is not None:
                actions.append(action)
            if not cycle:
                break
    return actions


show_doc(parse_senate_bill_status)

---

### parse_senate_bill_status

>      parse_senate_bill_status
>                                (status:legisph.senate.models.SenateBill.Senate
>                                BillStatus, classes:list)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| status | SenateBillStatus | Senate Bill Status to parse into a subclass |
| classes | list | List of classes through which to cycle |

In [119]:
parse_senate_bill_status(
    SenateBill.SenateBillStatus(
        date=datetime.date(2022, 10, 7), item="Pending in the Committee"
    ),
    classes=[PendingInCommittee],
)

[PendingInCommittee(date=datetime.date(2022, 10, 7), item='Pending in the Committee', name='Pending in Committee')]

## Parsing All Statuses

We then proceed to define all the remaining statuses below:


### Introduced

In [108]:
# | export
class Introduced(SenateBill.SenateBillStatus):
    """
    No matter where a legislative proposal originates, it can be introduced only by a
    member of Congress. In the Senate, a member may introduce any of several types of
    bills and resolutions by filing it with the Office of the Secretary.

    [Source](https://legacy.senate.gov.ph/about/legpro.asp#Introduction_of_Bills)
    """

    name: str = "Introduced by a Senator"
    senator: Senator

    @classmethod
    def parse(cls, h):
        if h.item.startswith("Introduced by Senator "):
            return (
                cls(
                    **h.dict(),
                    senator=Senator(
                        name=(
                            h.item.replace("Introduced by Senator ", "").replace(
                                ";", ""
                            )
                        )
                    )
                ),
                True,
            )
        return (None, True)

In [109]:
test_parsing(Introduced, "Introduced by Senator JINGGOY P. EJERCITO-ESTRADA;")

(Introduced(date=datetime.date(2022, 10, 11), item='Introduced by Senator JINGGOY P. EJERCITO-ESTRADA;', name='Introduced by a Senator', senator=Senator(name='JINGGOY P. EJERCITO-ESTRADA')),
 True)

### First Reading

In [12]:
# | export
class FirstReading(SenateBill.SenateBillStatus):
    """
    Once a measure has been introduced and given a number, it is read and referred to an
    appropriate committee. It must be noted that during the reading of the bill, only
    the title and the author is read on the floor. The Senate President is responsible
    for referring bills introduced to appropriate committees.

    The jurisdictions of the Standing Committees are spelled out in Rule X, Section 13
    of the Rules of the Senate. For example, if a bill involves matters relating to
    agriculture, food production and agri-business, it must be referred to the
    Committee on Agriculture and Food.

    [Source](https://legacy.senate.gov.ph/about/legpro.asp#Bill_Referrals)
    """

    name: str = "Read on First Reading and Referred to Committee"
    committees: List[SenateCommittee]

    @classmethod
    def parse(cls, h):
        slug1 = "Read on First Reading and Referred to the Committee on "
        slug2 = "Read on First Reading and Referred to the Committee(s) on "
        slug3 = "Read on First Reading and referred to the Committee(s) on "
        slug4 = "Read on First Reading and Referred to the Committees on "
        if h.item.startswith(slug1):
            committee = SenateCommittee(name=h.item.replace(slug1, "").replace(";", ""))
            return (cls(**h.dict(), committees=[committee]), True)
        if (
            h.item.startswith(slug2)
            or h.item.startswith(slug3)
            or h.item.startswith(slug4)
        ):
            committees = h.item.replace(slug2, "").replace(slug3, "").replace(slug4, "")
            committees = re.split("[ ]*and[ ]*|[ ]*;[ ]*", committees)
            committees = [
                SenateCommittee(name=name) for name in committees if name != ""
            ]
            return (cls(**h.dict(), committees=committees), True)
        return (None, True)

In [13]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee on JUSTICE AND HUMAN RIGHTS;",
)

(FirstReading(date=datetime.date(2022, 10, 11), item='Read on First Reading and Referred to the Committee on JUSTICE AND HUMAN RIGHTS;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='JUSTICE AND HUMAN RIGHTS')]),
 True)

In [14]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 11), item='Read on First Reading and Referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='EDUCATION, ARTS AND CULTURE'), SenateCommittee(name='FINANCE')]),
 True)

In [15]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee(s) on LOCAL GOVERNMENT; CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 11), item='Read on First Reading and Referred to the Committee(s) on LOCAL GOVERNMENT; CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='LOCAL GOVERNMENT'), SenateCommittee(name='CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION'), SenateCommittee(name='FINANCE')]),
 True)

In [16]:
test_parsing(
    FirstReading,
    "Read on First Reading and referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 11), item='Read on First Reading and referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='EDUCATION, ARTS AND CULTURE'), SenateCommittee(name='FINANCE')]),
 True)

### Organizational Briefing

In [17]:
# | export
class OrganizationalBriefing(SenateBill.SenateBillStatus):
    name: str = "Organizational Briefing"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted ORGANIZATIONAL MEETING/BRIEFING;":
            return (cls(**h.dict()), False)
        return (None, True)

In [18]:
test_parsing(OrganizationalBriefing, "Conducted ORGANIZATIONAL MEETING/BRIEFING;")

(OrganizationalBriefing(date=datetime.date(2022, 10, 11), item='Conducted ORGANIZATIONAL MEETING/BRIEFING;', name='Organizational Briefing'),
 False)

### Pending in Committee

In [19]:
# | export
class PendingInCommittee(SenateBill.SenateBillStatus):
    """
    The standing committees of the Senate, operating as “little legislatures,” determine
    the fate of most proposals. There are committee hearings scheduled to discuss the
    bills referred. Committee members and staff frequently are experts in the subjects
    under their jurisdiction, and it is at the committee stage that a bill comes under
    the sharpest scrutiny. If a measure is to be substantially revised, the revision
    usually occurs at the committee level.

    [Source](https://legacy.senate.gov.ph/about/legpro.asp#In_Committee)
    """

    name: str = "Pending in Committee"

    @classmethod
    def parse(cls, h):
        if h.item == "Pending in the Committee":
            return (cls(**h.dict()), False)
        return (None, True)

In [20]:
test_parsing(PendingInCommittee, "Pending in the Committee")

(PendingInCommittee(date=datetime.date(2022, 10, 11), item='Pending in the Committee', name='Pending in Committee'),
 False)

### Committee Proceedings

In [21]:
# | export
class CommitteeProceedings(SenateBill.SenateBillStatus):
    """
    The standing committees of the Senate, operating as “little legislatures,”
    determine the fate of most proposals. There are committee hearings scheduled to
    discuss the bills referred. Committee members and staff frequently are experts in
    the subjects under their jurisdiction, and it is at the committee stage that a bill
    comes under the sharpest scrutiny. If a measure is to be substantially revised, the
    revision usually occurs at the committee level.

    [Source](https://legacy.senate.gov.ph/about/legpro.asp#In_Committee)
    """

    name: str = "Conducted Committee Proceedings"
    joint: bool

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted COMMITTEE MEETINGS/HEARINGS;":
            return (cls(**h.dict(), joint=False), False)
        if h.item == "Conducted JOINT COMMITTEE MEETINGS/HEARINGS;":
            return (cls(**h.dict(), joint=True), False)
        return (None, True)

In [22]:
test_parsing(CommitteeProceedings, "Conducted COMMITTEE MEETINGS/HEARINGS;")

(CommitteeProceedings(date=datetime.date(2022, 10, 11), item='Conducted COMMITTEE MEETINGS/HEARINGS;', name='Conducted Committee Proceedings', joint=False),
 False)

In [23]:
test_parsing(CommitteeProceedings, "Conducted JOINT COMMITTEE MEETINGS/HEARINGS;")

(CommitteeProceedings(date=datetime.date(2022, 10, 11), item='Conducted JOINT COMMITTEE MEETINGS/HEARINGS;', name='Conducted Committee Proceedings', joint=True),
 False)

### Committee Consultation

In [136]:
# | export
class CommitteeConsultation(SenateBill.SenateBillStatus):
    """
    Committees may perform stakeholder consultations or ocular inspections to increase
    their understanding of the right form of legislation.
    """

    name: str = "Conducted Committee Consultation or Ocular"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted CONSULTATION/OCULAR;":
            return (cls(**h.dict()), False)
        return (None, True)

In [137]:
test_parsing(CommitteeConsultation, "Conducted CONSULTATION/OCULAR;")

(CommitteeConsultation(date=datetime.date(2022, 10, 11), item='Conducted CONSULTATION/OCULAR;', name='Conducted Committee Consultation or Ocular'),
 False)

### Archived

In [24]:
# | export
class Archived(SenateBill.SenateBillStatus):
    name: str = "Archived"

    @classmethod
    def parse(cls, h):
        if h.item == "SENT TO THE ARCHIVES.":
            return (cls(**h.dict()), False)
        if h.item == "Sent to the Archives":
            return (cls(**h.dict()), False)
        return (None, True)

In [25]:
test_parsing(Archived, "SENT TO THE ARCHIVES.")

(Archived(date=datetime.date(2022, 10, 11), item='SENT TO THE ARCHIVES.', name='Archived'),
 False)

### Committee Report Calendared For Ordinary Business

In [26]:
# | export
class CommitteeReportCalendaredForOrdinaryBusiness(SenateBill.SenateBillStatus):
    name: str = "Committe Report Calendared for Ordinary Business"

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Committee Report Calendared for Ordinary Business;",
            "Committee Report calendared for Ordinary Business;",
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [27]:
test_parsing(
    CommitteeReportCalendaredForOrdinaryBusiness,
    "Committee Report Calendared for Ordinary Business;",
)

(CommitteeReportCalendaredForOrdinaryBusiness(date=datetime.date(2022, 10, 11), item='Committee Report Calendared for Ordinary Business;', name='Committe Report Calendared for Ordinary Business'),
 False)

### Technical Working Group

In [28]:
# | export
class TechnicalWorkingGroup(SenateBill.SenateBillStatus):
    """
    A technical working group is an ad hoc group of experts on a particular topic who
    work together on specific goals.
    """

    name: str = "Conducted a Technical Working Group"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted TECHNICAL WORKING GROUP;":
            return (cls(**h.dict()), False)
        return (None, True)

In [29]:
test_parsing(TechnicalWorkingGroup, "Conducted TECHNICAL WORKING GROUP;")

(TechnicalWorkingGroup(date=datetime.date(2022, 10, 11), item='Conducted TECHNICAL WORKING GROUP;', name='Conducted a Technical Working Group'),
 False)

### Submitted Committee Report

In [30]:
# | export
class SubmittedCommitteeReport(SenateBill.SenateBillStatus):
    """
    A committee report describes the purpose and scope of the bill, explains any
    committee amendments, indicates proposed changes in existing law and such other
    materials that are relevant. Moreover, reports are numbered in the order in which
    they are filed and printed.
    """

    name: str = "Submitted Committee Report"

    @classmethod
    def parse(cls, h):
        if h.item == "Submitted said Committee Report to the Senate;":
            return (cls(**h.dict()), False)
        return (None, True)

### Consolidated or Substituted in Committee Report

In [152]:
# | export
class ConsolidatedOrSubstitutedInCommitteeReport(SenateBill.SenateBillStatus):
    """
    In the Committee Report, a bill is combined with another bill or substituted with
    another Senate Bill, effectively ending the life of this Senate Bill and continuing
    it in another one.
    """

    name: str = "Consolidated or Substituted in Committee Report"
    substitute_bill: Optional[str]
    committee_report: Optional[str]

    @classmethod
    def parse(cls, h):
        if h.item == "Consolidated/Substituted in the Committee Report":
            return (cls(**h.dict()), False)
        slug1 = "SUBSTITUTED BY "
        slug2 = " UNDER C.R. NO. "
        if h.item.startswith(slug1) and h.item.find(slug2) > -1:
            bill, report = h.item.split(slug2)
            bill = bill.replace(slug1, "")
            report = report.replace(".", "")
            return (
                cls(**h.dict(), substitute_bill=bill, committee_report=report),
                True,
            )
        slug3 = " UNDER COMMITTEE REPORT NO. "
        if h.item.startswith(slug1) and h.item.find(slug3) > -1:
            bill, report = h.item.split(slug3)
            bill = bill.replace(slug1, "")
            report = report.replace(".", "")
            return (
                cls(**h.dict(), substitute_bill=bill, committee_report=report),
                True,
            )
        if h.item.startswith(slug1):
            bill = h.item.replace(slug1, "").replace(".", "")
            return (
                cls(**h.dict(), substitute_bill=bill),
                True,
            )
        return (None, True)

In [142]:
test_parsing(
    ConsolidatedOrSubstitutedInCommitteeReport,
    "Consolidated/Substituted in the Committee Report",
)

(ConsolidatedOrSubstitutedInCommitteeReport(date=datetime.date(2022, 10, 11), item='Consolidated/Substituted in the Committee Report', name='Consolidated or Substituted in Committee Report', substitute_bill=None, committee_report=None),
 False)

In [147]:
test_parsing(
    ConsolidatedOrSubstitutedInCommitteeReport,
    "SUBSTITUTED BY SBN-2712 UNDER C.R. NO. 117.",
)

(ConsolidatedOrSubstitutedInCommitteeReport(date=datetime.date(2022, 10, 11), item='SUBSTITUTED BY SBN-2712 UNDER C.R. NO. 117.', name='Consolidated or Substituted in Committee Report', substitute_bill='SBN-2712', committee_report='117'),
 False)

In [149]:
test_parsing(
    ConsolidatedOrSubstitutedInCommitteeReport,
    "SUBSTITUTED BY SBN-1592 UNDER COMMITTEE REPORT NO. 164.",
)

(ConsolidatedOrSubstitutedInCommitteeReport(date=datetime.date(2022, 10, 11), item='SUBSTITUTED BY SBN-1592 UNDER COMMITTEE REPORT NO. 164.', name='Consolidated or Substituted in Committee Report', substitute_bill='SBN-1592', committee_report='164'),
 False)

### Approved on Second Reading

In [131]:
# | export
class ApprovedOnSecondReading(SenateBill.SenateBillStatus):
    name: str = "Approved On Second Reading"
    with_amendments: bool
    considered: Optional[str]

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Approved on Second Reading with Amendments;",
            "Approved on Second Reading with amendments;",
        ):
            return (cls(**h.dict(), with_amendments=True), False)
        if h.item == "Approved on Second Reading without Amendment;":
            return (cls(**h.dict(), with_amendments=False), False)
        slug1 = "Approved on Second Reading with Amendments, taking into consideration "
        if h.item.startswith(slug1):
            considered = h.item.replace(slug1, "")
            return (cls(**h.dict(), with_amendments=True, considered=considered), True)
        return (None, True)

In [132]:
test_parsing(ApprovedOnSecondReading, "Approved on Second Reading with Amendments;")

(ApprovedOnSecondReading(date=datetime.date(2022, 10, 11), item='Approved on Second Reading with Amendments;', name='Approved On Second Reading', with_amendments=True, considered=None),
 False)

In [35]:
test_parsing(ApprovedOnSecondReading, "Approved on Second Reading without Amendment;")

(ApprovedOnSecondReading(date=datetime.date(2022, 10, 11), item='Approved on Second Reading without Amendment;', name='Approved On Second Reading', with_amendments=False, considered=None),
 False)

### Transferred to Calendar for Special Order

In [36]:
# | export
class TransferredToCalendarForSpecialOrder(SenateBill.SenateBillStatus):
    name: str = "Transferred to Calendar for Special Order"

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;",
            "Transferred from Ordinary Business to Special Order;",
            "Assigned for Consideration by Special Order;",
            "Assigned for consideration by Special Order;",
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [37]:
test_parsing(
    TransferredToCalendarForSpecialOrder,
    "Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;",
)

(TransferredToCalendarForSpecialOrder(date=datetime.date(2022, 10, 11), item='Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;', name='Transferred to Calendar for Special Order'),
 False)

### Period of Interpellation Closed

In [150]:
# | export
class PeriodOfInterpellationClosed(SenateBill.SenateBillStatus):
    name: str = "Period of Interpellation Closed"

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Period of interpellation closed;",
            "Period of interpellations closed;",
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [39]:
test_parsing(PeriodOfInterpellationClosed, "Period of interpellation closed;")

(PeriodOfInterpellationClosed(date=datetime.date(2022, 10, 11), item='Period of interpellation closed;', name='Period of Interpellation Closed'),
 False)

### Sponsorship Speech

In [40]:
# | export
class SponsorshipSpeech(SenateBill.SenateBillStatus):
    name: str = "Sponsorship Speech"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug1 = "Sponsorship speech of Senator(s) "
        if h.item.startswith(slug1):
            senators = (
                h.item.replace(slug1, "")
                .replace(" on the Conference Committee Report;", "")
                .replace(" on the conference committee report;", "")
            )
            senators = [Senator(name=name) for name in senators.split(" and ")]
            return (cls(**h.dict(), senators=senators), True)
        slug2 = "Sponsorship speech on the Conference Committee Report of Senator "
        if h.item.startswith(slug2):
            senators = [Senator(name=h.item.replace(slug2, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug3 = "Sponsorship speech on the conference committee report of Senator "
        if h.item.startswith(slug3):
            senators = [Senator(name=h.item.replace(slug3, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug4 = "Sponsorship speech of Senator "
        if h.item.startswith(slug4):
            senators = [
                Senator(
                    name=(
                        h.item.replace(slug4, "")
                        .replace(";", "")
                        .replace(" on the conference committee report", "")
                        .replace(" on the Conference Committee Report", "")
                    )
                )
            ]
            return (cls(**h.dict(), senators=senators), True)
        slug5 = "Sponsorship speech delivered by Senator "
        if h.item.startswith(slug5):
            senators = [Senator(name=h.item.replace(slug5, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug6 = "Sponsorship Speech on the Conference Committee Report of Senator "
        slug7 = "Sponsorship speech on the  Conference Committee Report of Senator "
        if h.item.startswith(slug6) or h.item.startswith(slug7):
            senators = [
                Senator(
                    name=(h.item.replace(slug6, "").replace(slug7, "").replace(";", ""))
                )
            ]
            return (cls(**h.dict(), senators=senators), True)
        slug8 = "Co-sponsorship speech of Senator "
        if h.item.startswith(slug8):
            senators = [Senator(name=h.item.replace(slug8, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        return (None, True)

In [41]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech of Senator(s) BENIGNO S. AQUINO III and JINGGOY P. EJERCITO-ESTRADA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech of Senator(s) BENIGNO S. AQUINO III and JINGGOY P. EJERCITO-ESTRADA;', name='Sponsorship Speech', senators=[Senator(name='BENIGNO S. AQUINO III'), Senator(name='JINGGOY P. EJERCITO-ESTRADA;')]),
 True)

In [42]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech of Senator(s) EDGARDO J. ANGARA on the Conference Committee Report;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech of Senator(s) EDGARDO J. ANGARA on the Conference Committee Report;', name='Sponsorship Speech', senators=[Senator(name='EDGARDO J. ANGARA')]),
 True)

In [43]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech on the Conference Committee Report of Senator JOEL VILLANUEVA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech on the Conference Committee Report of Senator JOEL VILLANUEVA;', name='Sponsorship Speech', senators=[Senator(name='JOEL VILLANUEVA')]),
 True)

In [44]:
test_parsing(
    SponsorshipSpeech,
    'Sponsorship speech on the conference committee report of Senator EMMANUEL "MANNY" D. PACQUIAO;',
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech on the conference committee report of Senator EMMANUEL "MANNY" D. PACQUIAO;', name='Sponsorship Speech', senators=[Senator(name='EMMANUEL "MANNY" D. PACQUIAO')]),
 True)

In [45]:
test_parsing(
    SponsorshipSpeech,
    'Sponsorship speech of Senator RONALD "BATO" DELA ROSA on the conference committee report;',
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech of Senator RONALD "BATO" DELA ROSA on the conference committee report;', name='Sponsorship Speech', senators=[Senator(name='RONALD "BATO" DELA ROSA')]),
 True)

In [46]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech delivered by Senator SONNY ANGARA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship speech delivered by Senator SONNY ANGARA;', name='Sponsorship Speech', senators=[Senator(name='SONNY ANGARA')]),
 True)

In [47]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship Speech on the Conference Committee Report of Senator RISA HONTIVEROS;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 11), item='Sponsorship Speech on the Conference Committee Report of Senator RISA HONTIVEROS;', name='Sponsorship Speech', senators=[Senator(name='RISA HONTIVEROS')]),
 True)

### Sponsorship Speech Inserted into Record

In [48]:
# | export
class SponsorshipSpeechInsertedIntoRecord(SenateBill.SenateBillStatus):
    name: str = "Sponsorship Speech Inserted into Record"

    @classmethod
    def parse(cls, h):
        if h.item == "Sponsorship speech inserted into the record;":
            return (cls(**h.dict()), False)
        return (None, True)

In [49]:
test_parsing(
    SponsorshipSpeechInsertedIntoRecord,
    "Sponsorship speech inserted into the record;",
)

(SponsorshipSpeechInsertedIntoRecord(date=datetime.date(2022, 10, 11), item='Sponsorship speech inserted into the record;', name='Sponsorship Speech Inserted into Record'),
 False)

### Approved on Third Reading

In [133]:
# | export
class ApprovedOnThirdReading(SenateBill.SenateBillStatus):
    name: str = "Approved on Third Reading"

    @classmethod
    def parse(cls, h):
        if h.item in ("Approved on Third Reading;", "Approved on Third Reading"):
            return (cls(**h.dict()), False)
        return (None, True)

In [51]:
test_parsing(
    ApprovedOnThirdReading,
    "Approved on Third Reading;",
)

(ApprovedOnThirdReading(date=datetime.date(2022, 10, 11), item='Approved on Third Reading;', name='Approved on Third Reading'),
 False)

### Sent to the House of Representatives

In [52]:
# | export
class SentToHouseOfRepresentatives(SenateBill.SenateBillStatus):
    name: str = "Sent to the House of Representatives"

    @classmethod
    def parse(cls, h):
        if h.item == "Sent to the House of Representatives requesting for concurrence;":
            return (cls(**h.dict()), False)
        return (None, True)

In [53]:
test_parsing(
    SentToHouseOfRepresentatives,
    "Sent to the House of Representatives requesting for concurrence;",
)

(SentToHouseOfRepresentatives(date=datetime.date(2022, 10, 11), item='Sent to the House of Representatives requesting for concurrence;', name='Sent to the House of Representatives'),
 False)

### Individual amendments opened

In [54]:
# | export
class IndividualAmendmentsOpened(SenateBill.SenateBillStatus):
    name: str = "Individual amendments opened"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of individual amendments;":
            return (cls(**h.dict()), False)
        if h.item == "Period of amendment closed;":
            return (cls(**h.dict()), True)
        return (None, True)

In [55]:
test_parsing(
    IndividualAmendmentsOpened,
    "Period of individual amendments;",
)

(IndividualAmendmentsOpened(date=datetime.date(2022, 10, 11), item='Period of individual amendments;', name='Individual amendments opened'),
 False)

### Individual amendments closed

In [56]:
# | export
class IndividualAmendmentsClosed(SenateBill.SenateBillStatus):
    name: str = "Individual amendments closed"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of individual amendments closed;":
            return (cls(**h.dict()), False)
        if h.item == "Period of amendments closed;":
            return (cls(**h.dict()), True)
        return (None, True)

In [57]:
test_parsing(
    IndividualAmendmentsClosed,
    "Period of individual amendments closed;",
)

(IndividualAmendmentsClosed(date=datetime.date(2022, 10, 11), item='Period of individual amendments closed;', name='Individual amendments closed'),
 False)

### Committee amendments opened

In [58]:
# | export
class CommitteeAmendmentsOpened(SenateBill.SenateBillStatus):
    name: str = "Committee amendments opened"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of committee amendments;":
            return (cls(**h.dict()), False)
        if h.item == "Period of amendment closed;":
            return (cls(**h.dict()), True)
        return (None, True)

In [59]:
test_parsing(
    CommitteeAmendmentsOpened,
    "Period of committee amendments;",
)

(CommitteeAmendmentsOpened(date=datetime.date(2022, 10, 11), item='Period of committee amendments;', name='Committee amendments opened'),
 False)

### Committee amendments closed

In [60]:
# | export
class CommitteeAmendmentsClosed(SenateBill.SenateBillStatus):
    name: str = "Committee amendments closed"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of committee amendments closed;":
            return (cls(**h.dict()), False)
        if h.item == "Period of amendments closed;":
            return (cls(**h.dict()), True)
        return (None, True)

In [61]:
test_parsing(
    CommitteeAmendmentsClosed,
    "Period of committee amendments closed;",
)

(CommitteeAmendmentsClosed(date=datetime.date(2022, 10, 11), item='Period of committee amendments closed;', name='Committee amendments closed'),
 False)

### Printed copies distributed to Senators

In [62]:
# | export
class CopiesDistributed(SenateBill.SenateBillStatus):
    name: str = "Copies Distributed to Senators"

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Printed copies were distributed to the Senators;",
            "Printed Copies were distributed to the Senators;",
            "Printed copies were electronically distributed to the Senators;",
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [63]:
test_parsing(
    CopiesDistributed,
    "Printed copies were distributed to the Senators;",
)

(CopiesDistributed(date=datetime.date(2022, 10, 11), item='Printed copies were distributed to the Senators;', name='Copies Distributed to Senators'),
 False)

### Consolidated with Approved Bill

In [64]:
# | export
class ConsolidatedWithApprovedBill(SenateBill.SenateBillStatus):
    name: str = "Consolidated with Approved Bill"

    @classmethod
    def parse(cls, h):
        if h.item == "Consolidated with Approved Bill":
            return (cls(**h.dict()), False)
        return (None, True)

In [65]:
test_parsing(
    ConsolidatedWithApprovedBill,
    "Consolidated with Approved Bill",
)

(ConsolidatedWithApprovedBill(date=datetime.date(2022, 10, 11), item='Consolidated with Approved Bill', name='Consolidated with Approved Bill'),
 False)

### Approved by the President

In [66]:
# | export
class ApprovedByPresident(SenateBill.SenateBillStatus):
    name: str = "Approved by the President"

    @classmethod
    def parse(cls, h):
        if h.item == "Approved by the President of the Philippines":
            return (cls(**h.dict()), False)
        # if h.item.startswith("Approved and signed into law"):
        return (None, True)

In [67]:
test_parsing(
    ApprovedByPresident,
    "Approved by the President of the Philippines",
)

(ApprovedByPresident(date=datetime.date(2022, 10, 11), item='Approved by the President of the Philippines', name='Approved by the President'),
 False)

### Pending Second Reading

In [68]:
# | export
class PendingSecondReading(SenateBill.SenateBillStatus):
    name: str = "Pending Second Reading"
    type: str

    @classmethod
    def parse(cls, h):
        if h.item.startswith("Pending Second Reading, "):
            type = h.item.replace("Pending Second Reading, ", "")
            return (cls(**h.dict(), type=type), False)
        return (None, True)

In [69]:
test_parsing(PendingSecondReading, "Pending Second Reading, Ordinary Business")

(PendingSecondReading(date=datetime.date(2022, 10, 11), item='Pending Second Reading, Ordinary Business', name='Pending Second Reading', type='Ordinary Business'),
 False)

In [70]:
test_parsing(PendingSecondReading, "Pending Second Reading, Special Order")

(PendingSecondReading(date=datetime.date(2022, 10, 11), item='Pending Second Reading, Special Order', name='Pending Second Reading', type='Special Order'),
 False)

### Approved Conference Committee Report

In [71]:
# | export
class ApprovedConferenceCommitteeReport(SenateBill.SenateBillStatus):
    name: str = "Approved Conference Committee Report"

    @classmethod
    def parse(cls, h):
        if h.item == "Conference Committee Report Approved by the Senate;":
            return (cls(**h.dict()), False)
        return (None, True)

In [72]:
test_parsing(
    ApprovedConferenceCommitteeReport,
    "Conference Committee Report Approved by the Senate;",
)

(ApprovedConferenceCommitteeReport(date=datetime.date(2022, 10, 11), item='Conference Committee Report Approved by the Senate;', name='Approved Conference Committee Report'),
 False)

### Interpellation

In [73]:
# | export
class Interpellation(SenateBill.SenateBillStatus):
    name: str = "Interpellation"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug = "Interpellation of Senator "
        if h.item.startswith(slug):
            senators = [Senator(name=h.item.replace(slug, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        return (None, True)

In [74]:
test_parsing(
    Interpellation,
    "Interpellation of Senator FRANKLIN M. DRILON;",
)

(Interpellation(date=datetime.date(2022, 10, 11), item='Interpellation of Senator FRANKLIN M. DRILON;', name='Interpellation', senators=[Senator(name='FRANKLIN M. DRILON')]),
 True)

### Pending in the House of Representatives

In [75]:
# | export
class PendingInHouseOfRepresentatives(SenateBill.SenateBillStatus):
    name: str = "Pending in the House of Representatives"

    @classmethod
    def parse(cls, h):
        if h.item == "Pending in the House of Representatives":
            return (cls(**h.dict()), False)
        return (None, True)

In [76]:
test_parsing(
    PendingInHouseOfRepresentatives,
    "Pending in the House of Representatives",
)

(PendingInHouseOfRepresentatives(date=datetime.date(2022, 10, 11), item='Pending in the House of Representatives', name='Pending in the House of Representatives'),
 False)

### Withdrawn

In [114]:
# | export
class Withdrawn(SenateBill.SenateBillStatus):
    name: str = "Withdrawn"

    @classmethod
    def parse(cls, h):
        if h.item in ("Withdrawn", "WITHDRAWN.", "Withdrawn;"):
            return (cls(**h.dict()), False)
        return (None, True)

In [78]:
test_parsing(
    Withdrawn,
    "Withdrawn",
)

(Withdrawn(date=datetime.date(2022, 10, 11), item='Withdrawn', name='Withdrawn'),
 False)

### Sponsored

In [79]:
# | export
class Sponsored(SenateBill.SenateBillStatus):
    """
    When a Senator decides to become a proponent of a bill, they sponsor it. They decide
    when debate is ended on a bill and also delivers a sponsorship speech. A bill can
    have multiple sponsors and Senators may become co-sponsors at any point in the
    process.
    """

    name: str = "Sponsored"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug1 = "Sponsor:  Senator "
        slug2 = "Sponsor: Senator "
        slug3 = "Sponsor: Sen. "
        if (
            h.item.startswith(slug1)
            or h.item.startswith(slug2)
            or h.item.startswith(slug3)
        ):
            senators = [
                Senator(
                    name=(
                        h.item.replace(slug1, "")
                        .replace(slug2, "")
                        .replace(slug3, "")
                        .replace(";", "")
                    )
                )
            ]
            return (cls(**h.dict(), senators=senators), True)
        slug4 = "Sponsor: Senators "
        if h.item.startswith(slug4):
            senators = h.item.replace(slug4, "").replace(";", "")
            senators = [Senator(name=s.strip()) for s in senators.split(" and ")]
            return (cls(**h.dict(), senators=senators), True)
        return (None, True)

In [80]:
test_parsing(
    Sponsored,
    "Sponsor: Senator WIN GATCHALIAN;",
)

(Sponsored(date=datetime.date(2022, 10, 11), item='Sponsor: Senator WIN GATCHALIAN;', name='Sponsored', senators=[Senator(name='WIN GATCHALIAN')]),
 True)

In [81]:
test_parsing(
    Sponsored,
    'Sponsor: Senators  EMMANUEL "MANNY" D. PACQUIAO and JUAN EDGARDO "SONNY" M. ANGARA;',
)

(Sponsored(date=datetime.date(2022, 10, 11), item='Sponsor: Senators  EMMANUEL "MANNY" D. PACQUIAO and JUAN EDGARDO "SONNY" M. ANGARA;', name='Sponsored', senators=[Senator(name='EMMANUEL "MANNY" D. PACQUIAO'), Senator(name='JUAN EDGARDO "SONNY" M. ANGARA')]),
 True)

### Votes In Favor

In [115]:
# | export
class VotesInFavor(SenateBill.SenateBillStatus):
    """
    Senators have voted and some senators are in favor.
    """

    name: str = "Votes In Favor"
    vote: str = "In Favor"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug = "In favor:"
        if h.item.startswith(slug):
            senators = h.item.replace(slug, "").replace(";", "").strip()
            senators = re.sub("\([1-9]+\)", "", senators)
            senators = re.sub("Senator[s]", "", senators)
            split = re.split(",|and", senators)
            senators = (
                [Senator(name=s.strip()) for s in split]
                if senators.lower().strip() not in ("(none)", "n o n e")
                else []
            )
            return (cls(**h.dict(), senators=senators), False)
        return (None, True)

In [117]:
test_parsing(
    VotesInFavor,
    'In favor: (23) Senators SONNY ANGARA, MARIA LOURDES NANCY S. BINAY, PIA S. CAYETANO, RONALD "BATO" DELA ROSA, FRANKLIN M. DRILON, WIN GATCHALIAN, CHRISTOPHER LAWRENCE T. GO, RICHARD J. GORDON, RISA HONTIVEROS, PANFILO "PING" M. LACSON, MANUEL "LITO" M. LAPID, IMEE R. MARCOS, EMMANUEL "MANNY" D. PACQUIAO, FRANCIS "KIKO" N. PANGILINAN, AQUILINO "KOKO" III PIMENTEL, GRACE POE, RALPH G. RECTO, RAMON BONG REVILLA JR., VICENTE C. SOTTO III, FRANCIS "TOL" N. TOLENTINO, JOEL VILLANUEVA, CYNTHIA A. VILLAR and JUAN MIGUEL "MIGZ" F. ZUBIRI;',
)

(VotesInFavor(date=datetime.date(2022, 10, 11), item='In favor: (23) Senators SONNY ANGARA, MARIA LOURDES NANCY S. BINAY, PIA S. CAYETANO, RONALD "BATO" DELA ROSA, FRANKLIN M. DRILON, WIN GATCHALIAN, CHRISTOPHER LAWRENCE T. GO, RICHARD J. GORDON, RISA HONTIVEROS, PANFILO "PING" M. LACSON, MANUEL "LITO" M. LAPID, IMEE R. MARCOS, EMMANUEL "MANNY" D. PACQUIAO, FRANCIS "KIKO" N. PANGILINAN, AQUILINO "KOKO" III PIMENTEL, GRACE POE, RALPH G. RECTO, RAMON BONG REVILLA JR., VICENTE C. SOTTO III, FRANCIS "TOL" N. TOLENTINO, JOEL VILLANUEVA, CYNTHIA A. VILLAR and JUAN MIGUEL "MIGZ" F. ZUBIRI;', name='Votes In Favor', vote='In Favor', senators=[Senator(name='SONNY ANGARA'), Senator(name='MARIA LOURDES NANCY S. BINAY'), Senator(name='PIA S. CAYETANO'), Senator(name='RONALD "BATO" DELA ROSA'), Senator(name='FRANKLIN M. DRILON'), Senator(name='WIN GATCHALIAN'), Senator(name='CHRISTOPHER LAWRENCE T. GO'), Senator(name='RICHARD J. GORDON'), Senator(name='RISA HONTIVEROS'), Senator(name='PANFILO "PING"

### Votes In Favor

In [105]:
# | export
class VotesAgainst(SenateBill.SenateBillStatus):
    """
    Senators have voted and some senators are in favor.
    """

    name: str = "Votes Against"
    vote: str = "Against"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug1 = "Against:"
        slug2 = "Against"
        if h.item.startswith(slug1) or h.item.startswith(slug2):
            senators = (
                h.item.replace(slug1, "")
                .replace(slug2, "")
                .replace(":", "")
                .replace(";", "")
                .strip()
            )
            senators = re.sub("\([1-9]+\)", "", senators)
            senators = re.sub("Senator[s]", "", senators)
            split = re.split(",|and", senators)
            senators = (
                [Senator(name=s.strip()) for s in split]
                if senators.lower().strip() not in ("(none)", "n o n e")
                else []
            )
            return (cls(**h.dict(), senators=senators), False)
        return (None, True)

In [104]:
test_parsing(
    VotesAgainst,
    "Against:         N o n e;",
)

(VotesAgainst(date=datetime.date(2022, 10, 11), item='Against:         N o n e;', name='Votes Against', vote='Against', senators=[]),
 False)

In [106]:
test_parsing(
    VotesAgainst,
    "Against  : (2) Senators Lim and Pimentel, Jr.;",
)

(VotesAgainst(date=datetime.date(2022, 10, 11), item='Against  : (2) Senators Lim and Pimentel, Jr.;', name='Votes Against', vote='Against', senators=[Senator(name='Lim'), Senator(name='Pimentel'), Senator(name='Jr.')]),
 False)

### Votes Abstained

In [89]:
# | export
class VotesAbstained(SenateBill.SenateBillStatus):
    """
    Senators have voted and some senators have abstained from voting.
    """

    name: str = "Votes Abstained"
    vote: str = "Abstained"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug1 = "Abstention:"
        slug2 = "Abstain:"
        slug3 = "Abstention"
        if (
            h.item.startswith(slug1)
            or h.item.startswith(slug2)
            or h.item.startswith(slug3)
        ):
            senators = (
                h.item.replace(slug1, "")
                .replace(slug2, "")
                .replace(slug3, "")
                .replace(";", "")
                .strip()
            )
            senators = re.sub("\([1-9]+\)", "", senators)
            split = re.split(",|and", senators)
            senators = (
                [Senator(name=s.strip()) for s in split]
                if senators.lower().strip() not in ("(none)", "n o n e")
                else []
            )
            return (cls(**h.dict(), senators=senators), False)
        return (None, True)

In [90]:
test_parsing(
    VotesAbstained,
    "Abstention:    (2) JINGGOY P. EJERCITO-ESTRADA and JUAN PONCE ENRILE;",
)

(VotesAbstained(date=datetime.date(2022, 10, 11), item='Abstention:    (2) JINGGOY P. EJERCITO-ESTRADA and JUAN PONCE ENRILE;', name='Votes Abstained', vote='Abstained', senators=[Senator(name='JINGGOY P. EJERCITO-ESTRADA'), Senator(name='JUAN PONCE ENRILE')]),
 False)

We do a test as to whether we can succesfully parse a very common combination of votes separated by newlines instead of being in different statuses:

In [113]:
parse_senate_bill_status(
    SenateBill.SenateBillStatus(
        date=datetime.date.today(),
        item="In favor: (22) Senators Angara, Aquino, Arroyo, Biazon, Alan Peter Cayetano, Pia Cayetano, Miriam Defensor Santiago, Jinggoy Ejercito Estrada, Enrile, Escudero, Gordon, Honasan, Lacson, Lapid, Legarda, M.A.Madrigal, Pangilinan, Pimentel, Jr., Revilla, Jr., Roxas, Villar, and Zubiri;\nAgainst:      N o n e;\nAbstention:  N o n e;",
    ),
    [VotesInFavor, VotesAbstained, VotesAgainst],
)

[VotesInFavor(date=datetime.date(2022, 10, 11), item='In favor: (22) Senators Angara, Aquino, Arroyo, Biazon, Alan Peter Cayetano, Pia Cayetano, Miriam Defensor Santiago, Jinggoy Ejercito Estrada, Enrile, Escudero, Gordon, Honasan, Lacson, Lapid, Legarda, M.A.Madrigal, Pangilinan, Pimentel, Jr., Revilla, Jr., Roxas, Villar, and Zubiri', name='Votes In Favor', vote='In Favor', senators=[Senator(name='Angara'), Senator(name='Aquino'), Senator(name='Arroyo'), Senator(name='Biazon'), Senator(name='Alan Peter Cayetano'), Senator(name='Pia Cayetano'), Senator(name='Miriam Defensor Santiago'), Senator(name='Jinggoy Ejercito Estrada'), Senator(name='Enrile'), Senator(name='Escudero'), Senator(name='Gordon'), Senator(name='Honasan'), Senator(name='Lacson'), Senator(name='Lapid'), Senator(name='Legarda'), Senator(name='M.A.Madrigal'), Senator(name='Pangilinan'), Senator(name='Pimentel'), Senator(name='Jr.'), Senator(name='Revilla'), Senator(name='Jr.'), Senator(name='Roxas'), Senator(name='Villa

### Lapsed into Law

In [125]:
# | export
class LapsedIntoLaw(SenateBill.SenateBillStatus):
    """
    When the President fails to sign a bill into law within a set amount of time, the
    bill is considered to have lapsed into law and will be as if he had approved it.
    """

    name: str = "Lapsed into Law"

    @classmethod
    def parse(cls, h):
        if h.item in ("Lapsed Into Law"):
            return (cls(**h.dict()), False)
        return (None, True)

In [126]:
test_parsing(LapsedIntoLaw, "Lapsed Into Law")

(LapsedIntoLaw(date=datetime.date(2022, 10, 11), item='Lapsed Into Law', name='Lapsed into Law'),
 False)

### Inquiry of the Chair

In [134]:
# | export
class InquiryOfChair(SenateBill.SenateBillStatus):
    name: str = "Inquiry of the Chair"

    @classmethod
    def parse(cls, h):
        if h.item in ("Inquiry of the Chair;"):
            return (cls(**h.dict()), False)
        return (None, True)

In [135]:
test_parsing(InquiryOfChair, "Inquiry of the Chair;")

(InquiryOfChair(date=datetime.date(2022, 10, 11), item='Inquiry of the Chair;', name='Inquiry of the Chair'),
 False)